In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from materials import get_materials_Cvec
from safe_module import distance

In [ ]:
write = False

In [ ]:
# histograms

def hist(betas, sigma, c_sigma):

    global axs, p, k

    mean = np.mean(betas)
    stdv = np.std(betas)

    bin_width = 0.1

    ax = axs[k]
    n, _, _ = ax.hist(betas, bins=np.arange(0, 60 + bin_width, bin_width), density=True)
    #ax.set_xlim(-0.1 * max(betas), 1.1 * max(betas))
    #ax.set_xlim(0, 40)
    ax.axvline(beta0,color='k')
    ax.axvline(mean,color='r')
    ax.axvline(mean-2*stdv,color='r',linestyle='--')
    ax.axvline(mean+2*stdv,color='r',linestyle='--')
    ax.set_title(r'$\beta_{{{}}}$ ($\mu$ = {:.1f}, 2$\sigma$ = {:.1f}), c_sigma = {:.1f}'.format(sigma, mean, 2*stdv, c_sigma),
                 fontsize = 30)
    ax.tick_params(axis='both', which='major', labelsize=30)

    p[k] = n[0]

In [ ]:
# Uncertainty analysis for WG100 (0.1 MPa) from Lokajicek et al. 2021

c_vecs = get_materials_Cvec(material="Lokajicek")
c_vec = c_vecs[4]
data_size = 10000
sigma = 'ISO'

In [ ]:
beta0 = distance(c_vec, 'ISO')[0]

In [ ]:
c_vec_stdvs = np.arange(0, 10.1, 0.2)
n_stdvs = len(c_vec_stdvs)

p = np.zeros(n_stdvs)
betas_mean = np.zeros(n_stdvs)

fig_depth = 5 * n_stdvs
fig, axs = plt.subplots(n_stdvs,1,figsize=(16,fig_depth))
for k, c_vec_stdv in enumerate(c_vec_stdvs):
    print(f'running iteration {k} ....')
    betas = np.empty(data_size)
    for i in range(data_size):
        c_vec_random = np.around( np.random.normal(c_vec, c_vec_stdv), 1 )
        result = distance(c_vec_random, sigma)
        betas[i] = result[0]

    hist(betas, sigma, c_vec_stdv)
    betas_mean[k] = np.mean(betas)

plt.subplots_adjust(hspace=0.5)
if write: plt.savefig('Lokajicek_uncertainty_3.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure()
plt.plot(c_vec_stdvs, p)
plt.xlabel('c_sigma')
plt.ylabel('p(beta=0)')
if write: plt.savefig('Lokajicek_uncertainty_1.png', bbox_inches='tight')
plt.show()

In [ ]:
def fit_data(x, y, degree, x_plot):
    p = np.polyfit(x, y, degree)
    fit = []
    for i, xp in enumerate(x_plot):
        fit.append(0)
        for j in range(degree+1):
            fit[i] = fit[i] + p[degree-j] * xp ** j
    return fit

degree = 1

x1 = c_vec_stdvs[5:31]
y1 = betas_mean[5:31]
x_plot1 = c_vec_stdvs
fit1 = fit_data(x1, y1, degree, x_plot1)

x2 = c_vec_stdvs[33:51]
y2 = betas_mean[33:51]
x_plot2 = c_vec_stdvs
fit2 = fit_data(x2, y2, degree, x_plot2)

plt.figure()
plt.plot(c_vec_stdvs, betas_mean, '-o', label='data')
plt.plot(x_plot1, fit1, 'r--', label='fit window 1')
plt.plot(x_plot2, fit2, 'k--', label='fit window 2')
plt.xlabel('c_sigma')
plt.ylabel('mean(beta)')
plt.legend()
if write: plt.savefig('Lokajicek_uncertainty_2.png', bbox_inches='tight')
plt.show()